In [3]:
import torch
from torch import nn
from torch.nn import init
import numpy as np
import sys
import d2lzh_pytorch as d2l
import torchvision
import torchvision.transforms as transforms

## 1.导入数据

In [7]:
#这里仍然和书中有差距，因为我没有把书中的函数放到d2l文件中，这里仍然是直接利用dataset和dataloader实现
batch_size = 256
mnist_train = torchvision.datasets.FashionMNIST(root = '~/DataSets/FashionMNIST',train = True,download = True,transform = transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST(root = '~/DataSets/FashionMNIST',train = False,download = True,transform = transforms.ToTensor())
if sys.platform.startswith('win'):
    num_workers = 0
else:
    num_workers = 4
train_iter = torch.utils.data.DataLoader(mnist_train,batch_size,shuffle=True,num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(mnist_test,batch_size,shuffle=True,num_workers=num_workers)

## 2.定义和初始化模型
在softmax部分的输入和输出都是全连接层，但是我们的数据集本质上是（batch_size,1,28,28)，所以要先转换格式，变成（batch_size,784）的大小

In [31]:
num_inputs = 784
num_outputs = 10
class LinearNet(nn.Module):
    def __init__(self,num_inputs,num_outputs):
        super(LinearNet,self).__init__()
        self.linear = nn.Linear(num_inputs,num_outputs)
    def forward(self,x):
        y = self.linear(x.view(x.shape[0],-1))
        return y
#模型实例化
net = LinearNet(num_inputs,num_outputs)
        

In [32]:
print(net)

LinearNet(
  (linear): Linear(in_features=784, out_features=10, bias=True)
)


In [33]:
#初始化参数
init.normal_(net.linear.weight,mean=0,std=0.01)
init.constant_(net.linear.bias,val=0)

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

## 3.定义交叉熵损失函数，和优化算法


In [34]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(),lr=0.1)

## 4.训练模型
因为原文这里放到了d2l里面，我这里有手动实现了一次方便熟悉

In [36]:
def train_ch3(net,train_iter,test_iter,loss,num_epochs,batch_size,params=None,lr =None,optimizer=None):
    for epoch in range(num_epochs):
        train_loss_sum,train_acc_sum,n=0.0,0.0,0
        for X,y in train_iter:
            y_hat = net(X)
            l = loss(y_hat,y).sum()
            #先梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            #反向传播
            l.backward()
            if optimizer is None:
                d2l.sgd(params,lr,batch_size)
            else:
                optimizer.step()
            train_loss_sum+=l
            train_acc_sum+=(y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = d2l.evaluate_accuracy(test_iter,net)
        print('epoch %d,loss%.4f,train acc %.3f,test acc %.3f'
              %(epoch+1,train_loss_sum/n,train_acc_sum/n,test_acc))
#因为前面在net那里进行了初始化参数，在loss这里进行了lr设置，所以这里直接设置为了None
train_ch3(net = net,train_iter=train_iter,test_iter=test_iter,loss=loss,num_epochs=num_epochs,batch_size=batch_size,params=None,lr=None,optimizer=optimizer)
      
    

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20128\3261789000.py:23: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\autograd\generated\python_variable_methods.cpp:836.)
  print('epoch %d,loss%.4f,train acc %.3f,test acc %.3f'


epoch 1,loss0.0022,train acc 0.814,test acc 0.798
epoch 2,loss0.0021,train acc 0.824,test acc 0.804
epoch 3,loss0.0020,train acc 0.831,test acc 0.818
epoch 4,loss0.0019,train acc 0.837,test acc 0.812
epoch 5,loss0.0019,train acc 0.840,test acc 0.825
